In [29]:
import pandas as pd
import re
import spacy
import numpy as np
import gzip
import gensim.downloader
import torch
from torch.utils.data import Dataset, DataLoader
import pickle

In [7]:
# train = pd.read_csv("./nlp-getting-started/train.csv")
# test = pd.read_csv("./nlp-getting-started/test.csv")

# train["isTrain"] = True
# test["isTrain"] = False

# full = pd.concat([train, test])
# # print(full)

# def clean_text(row):
#     clean = row["text"]
    
#     if len(row["tags"]) != 0:
#         for word in row["tags"]:
#             clean = clean.replace(word, "")
    
#     if len(row["links"]) != 0:
#         for word in row["links"]:
#             clean = clean.replace(word, "")
    
#     #only remove the # symbol
#     clean = clean.replace("#", "").replace("/", "").replace("(", "").replace(")", "")
    
#     return clean.strip()

# def get_at(row):
#     return re.findall("@[\w]+", row["text"])

# def get_http(row):
#     return re.findall("http[\:\/\.\w]+", row["text"])

# def get_hashtags(row):
#     return re.findall("#[\w]+", row["text"])

# def number_of_tags(row):
#     return len(row["tags"])

# def number_of_links(row):
#     return len(row["links"])

# def number_of_hashs(row):
#     return len(row["hashtags"])

# full["tags"] = full.apply(lambda row: get_at(row), axis = 1)
# full["links"] = full.apply(lambda row: get_http(row), axis = 1)
# full["hashtags"] = full.apply(lambda row: get_hashtags(row), axis = 1)

# full["number_of_tags"] = full.apply(lambda row: number_of_tags(row), axis = 1)
# full["number_of_links"] = full.apply(lambda row: number_of_links(row), axis = 1)
# full["number_of_hashs"] = full.apply(lambda row: number_of_hashs(row), axis = 1)

# full["clean_text"] = full.apply(lambda row: clean_text(row), axis = 1)
# full.sample(5)

/home/sour/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys


,id,isTrain,keyword,location,target,text,tags,links,hashtags,number_of_tags,number_of_links,number_of_hashs,clean_text
7331,10492,True,wildfire,Amsterdam | San Francisco,1.0,'Some hearths they burn like a wildfire' https...,[],[https://t.co/of3td6DGLb],[],0,1,0,'Some hearths they burn like a wildfire' by B...
3546,5069,True,famine,PanamÌÁ,0.0,Top 3:\n* Turn on the Darkness\n* King Redeem ...,[],[],[],0,0,0,Top 3:\n* Turn on the Darkness\n* King Redeem ...
5938,8480,True,screamed,pissing off antis,0.0,I SCREAMED 'WHATS A CHONCe' http://t.co/GXYivs...,[],[http://t.co/GXYivsWki7],[],0,1,0,I SCREAMED 'WHATS A CHONCe'
1632,5504,False,flames,NaN,NaN,This is gonna go down in flames ?? https://t.c...,[],[https://t.co/pjOG1LcYYV],[],0,1,0,This is gonna go down in flames ??
6752,9673,True,tornado,Midwest,1.0,(OK) Severe Thunderstorm Warning issued Augus...,[],[http://t.co/O8X4M5eR6b],[#okwx],0,1,1,OK Severe Thunderstorm Warning issued August ...


In [30]:
with gzip.open('./test.txt.gz', 'rt') as f:
    lines = [x.rstrip('\n') for x in f.readlines()]
    
    sentences = [[]]
    idx = 0
    for line in lines:
        if line != '':
            sentences[idx].append(line.split()[0])
        else:
            sentences.append([])
            idx += 1
            
#     print(sentences[:100])
    
    for sentence in sentences:
        for i, word in enumerate(sentence):
            if word == "'s":
                sentence[i-1] = sentence[i-1] + "'s"
                del sentence[i]
                
    print(sentences[:100])
    print(len(sentences))

[['Rockwell', 'International', "Corp.'s", 'Tulsa', 'unit', 'said', 'it', 'signed', 'a', 'tentative', 'agreement', 'extending', 'its', 'contract', 'with', 'Boeing', 'Co.', 'to', 'provide', 'structural', 'parts', 'for', "Boeing's", '747', 'jetliners', '.'], ['Rockwell', 'said', 'the', 'agreement', 'calls', 'for', 'it', 'to', 'supply', '200', 'additional', 'so-called', 'shipsets', 'for', 'the', 'planes', '.'], ['These', 'include', ',', 'among', 'other', 'parts', ',', 'each', "jetliner's", 'two', 'major', 'bulkheads', ',', 'a', 'pressure', 'floor', ',', 'torque', 'box', ',', 'fixed', 'leading', 'edges', 'for', 'the', 'wings', 'and', 'an', 'aft', 'keel', 'beam', '.'], ['Under', 'the', 'existing', 'contract', ',', 'Rockwell', 'said', ',', 'it', 'has', 'already', 'delivered', '793', 'of', 'the', 'shipsets', 'to', 'Boeing', '.'], ['Rockwell', ',', 'based', 'in', 'El', 'Segundo', ',', 'Calif.', ',', 'is', 'an', 'aerospace', ',', 'electronics', ',', 'automotive', 'and', 'graphics', 'concern', '.

In [31]:
nlp = spacy.load('en_core_web_sm')

sub_toks = []
sub_indices = []

new_sentences = []

for i, sentence in enumerate(sentences):
    sent = ' '.join(sentence)
#     print(sent)
    doc = nlp(sent)
    temp = [tok for tok in doc if (tok.dep_ == "nsubj")]
    if len(temp) == 1:
        if str(temp[0]) in sentence:
            new_sentences.append(sentence)
            sub_toks.append(temp[0])
            sub_indices.append(sentence.index(str(temp[0])))
            
    if i + 1 % 200 == 0:
        print('{} out of {} done'.format(i+1, len(sentences)))
    
print(sub_toks[:10])
print(sub_indices[:10])
print(new_sentences[:10])

[These, Rockwell, Carlucci, he, Meador, Meador, who, Shearson, Bank, bank]
[0, 0, 1, 3, 1, 12, 11, 0, 2, 6]
[['These', 'include', ',', 'among', 'other', 'parts', ',', 'each', "jetliner's", 'two', 'major', 'bulkheads', ',', 'a', 'pressure', 'floor', ',', 'torque', 'box', ',', 'fixed', 'leading', 'edges', 'for', 'the', 'wings', 'and', 'an', 'aft', 'keel', 'beam', '.'], ['Rockwell', ',', 'based', 'in', 'El', 'Segundo', ',', 'Calif.', ',', 'is', 'an', 'aerospace', ',', 'electronics', ',', 'automotive', 'and', 'graphics', 'concern', '.'], ['Mr.', 'Carlucci', ',', '59', 'years', 'old', ',', 'served', 'as', 'defense', 'secretary', 'in', 'the', 'Reagan', 'administration', '.'], ['In', 'January', ',', 'he', 'accepted', 'the', 'position', 'of', 'vice', 'chairman', 'of', 'Carlyle', 'Group', ',', 'a', 'merchant', 'banking', 'concern', '.'], ['Mr.', 'Meador', 'had', 'been', 'executive', 'vice', 'president', 'of', 'Balcor', '.'], ['In', 'addition', 'to', 'his', 'previous', 'real-estate', 'investment

In [32]:
print(len(new_sentences))
print(len(sub_toks))
print(len(sub_indices))

print(new_sentences[:10])
print(sub_toks[:10])
print(sub_indices[:10])

pickle.dump({'sentences': new_sentences, 'sub_indices': sub_indices}, open('subdata_test.pkl', 'wb'))

881
881
881
[['These', 'include', ',', 'among', 'other', 'parts', ',', 'each', "jetliner's", 'two', 'major', 'bulkheads', ',', 'a', 'pressure', 'floor', ',', 'torque', 'box', ',', 'fixed', 'leading', 'edges', 'for', 'the', 'wings', 'and', 'an', 'aft', 'keel', 'beam', '.'], ['Rockwell', ',', 'based', 'in', 'El', 'Segundo', ',', 'Calif.', ',', 'is', 'an', 'aerospace', ',', 'electronics', ',', 'automotive', 'and', 'graphics', 'concern', '.'], ['Mr.', 'Carlucci', ',', '59', 'years', 'old', ',', 'served', 'as', 'defense', 'secretary', 'in', 'the', 'Reagan', 'administration', '.'], ['In', 'January', ',', 'he', 'accepted', 'the', 'position', 'of', 'vice', 'chairman', 'of', 'Carlyle', 'Group', ',', 'a', 'merchant', 'banking', 'concern', '.'], ['Mr.', 'Meador', 'had', 'been', 'executive', 'vice', 'president', 'of', 'Balcor', '.'], ['In', 'addition', 'to', 'his', 'previous', 'real-estate', 'investment', 'and', 'asset-management', 'duties', ',', 'Mr.', 'Meador', 'takes', 'responsibility', 'for', 

In [46]:
print(list(gensim.downloader.info()['models'].keys()))

glove = gensim.downloader.load('glove-wiki-gigaword-200')

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [63]:
a = glove[['augusts', 'bye']]
print(a.shape)
print('Hi' in glove)

(2, 200)
False


In [64]:
class WordDataset(Dataset):
    
    def __init__(self, sfile):
        d = pickle.load(open(sfile, 'rb'))
        
        self.sentences = d['sentences']
        self.indices = d['sub_indices']
        
        for sent in self.sentences:
            for i, word in enumerate(sent):
                sent[i] = word.lower().replace("'", "")
                if sent[i] == '':
                    del sent[i]
        
        self.glove = gensim.downloader.load('glove-wiki-gigaword-200')
        
        self.sentence_embeddings = []
        
        for sent in self.sentences:
            temp = []
            for word in sent:
                if word in self.glove:
                    temp.append(self.glove[word])
            self.sentence_embeddings.append(np.array(temp))
            
    def __len__(self):
        return len(self.sentence_embeddings)
    
    def __getitem__(self, idx):
        return (self.sentences[idx], self.sentence_embeddings[idx], self.indices[idx])

In [65]:
train_dataset = WordDataset('subdata.pkl')
test_dataset = WordDataset('subdata_test.pkl')

In [68]:
print(train_dataset[1][1].shape)

(26, 200)
